# Differential Analysis Result Tables 

- [ ] short report about raw data and experiment design 
- [ ] describe analysis workflow and applied tools 
- [ ] describe results (plots, tables)
- [ ] final interpretation 

## meRIP-seq
Experiment design: Two replicates of samples for both treated with Decitabine drug and non-treated (
`T1.input.bam`
`T1.m6A.bam`
`T2.input.bam`
`T2.m6A.bam`
`U1.input.bam`
`U1.m6A.bam`
`U2.input.bam`
`U2.m6A.bam`
) used for meRIP-seq experiment. 

Aim: analyzing differential methylation of MeRIP-seq data

Tool: [RADAR](https://github.com/scottzijiezhang/RADAR)

#### Result: 
`meRIP-seq/hl60_delta_mtyl_table.txt`. 
_Columns_: 
- `chr`
- `start`
- `end`
- `name`
- `score`
- `strand`
- `thickStart`
- `thickEnd`
- `itemRgb`
- `blockCount`
- `blockSizes`
- `blockStarts`
- `logFC`
- `p_value`
- `ensembl`

## RNA-seq
Aim-1: analyzing differential experssion of RNA-seq data

Tool: [DESeq2](https://bioconductor.org/packages/release/bioc/html/DESeq2.html), [iPAGE](https://github.com/goodarzilab/PAGE)

#### Result: 
`RNA-seq/hl60-exp/hl60_exp_deseq2_norm.txt` - normalized counts 

`RNA-seq/hl60-exp/hl60_delta_exp_table.txt`
- `gene_id`
- `gene_name`
- `baseMean`
- `log2FC_6h`
- `pval_6h`
- `log2FC_72h`
- `pval_72h`
- `log2FC_120h`
- `pval_120h`

Aim-2: cell line consistency analysis 


## Ribo-seq

Aim: analyzing differential translation of Ribo-seq data

Tool: [Ribolog](https://github.com/goodarzilab/Ribolog)


#### Result:
`Ribo-seq/hl60_delta_te_table.txt`

- `transcript`
- `gene_id`
- `gene_name`
- `Estimate_treatmentDRUG`
- `Pr...z.._treatmentDRUG`
- `fdr_Pr...z.._treatmentDRUG`
- `qvalue_Pr...z.._treatmentDRUG`


## CRISPRi-screen
Aim: analyzing differential phenotype of CRISPRi-screen data

Tool: [ScreenProcessing](https://github.com/mhorlbeck/ScreenProcessing)

#### Result:
`CRISPRi-screen/hl60_exp1/DAC_processing_output_genetable_collapsed.txt`

`CRISPRi-screen/hl60_exp2/DAC_processing_output_genetable_collapsed.txt`



# Combined Analysis

- [ ] describe analysis purpose 
- [ ] describe results (plots, tables)
- [ ] final interpretation 

## hyper-geometric-test

## mtyl-enrichment

## intersectional-genes
